In [1]:

import numpy as np
import scipy.io

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Dropout, Flatten
from keras.layers.core import Permute, Reshape
from keras import backend as K

np.random.seed(2)



Using TensorFlow backend.


In [2]:

data = scipy.io.loadmat('data/PAMAP2_f')
print('including MLP/CNN/LSTM/ConvLSTM')
X_train0 = data['X_train']
X_valid0 = data['X_valid']
X_test0 = data['X_test']
y_train = data['y_train'].reshape(-1).astype(np.uint8)
y_valid = data['y_valid'].reshape(-1).astype(np.uint8)
y_test = data['y_test'].reshape(-1).astype(np.uint8)

print('merging validation set and test set into one')
X_valid0 = np.concatenate((X_valid0, X_test0), axis=0)
y_valid = np.concatenate((y_valid, y_test), axis=0)

num_classes = 12 # 12 classes for PAMAP2

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)



including MLP/CNN/LSTM/ConvLSTM
merging validation set and test set into one


In [3]:
print(y_train)

[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]


In [4]:


def _data_reshaping(X_tr, X_va, network_type):
    _, win_len, dim = X_tr.shape
    print(network_type)
    if network_type=='CNN' or network_type=='ConvLSTM':
        
        # make it into (frame_number, dimension, window_size, channel=1) for convNet
        X_tr = np.swapaxes(X_tr,1,2)
        X_va = np.swapaxes(X_va,1,2)

        X_tr = np.reshape(X_tr, (-1, dim, win_len, 1))
        X_va = np.reshape(X_va, (-1, dim, win_len, 1))
        
    if network_type=='MLP':
        print('MLP...')
        X_tr = np.reshape(X_tr, (-1, dim*win_len))
        X_va = np.reshape(X_va, (-1, dim*win_len))
    
    return X_tr, X_va


In [5]:

def model_variant(model, num_feat_map, dim, network_type):
    print(network_type)
    if network_type == 'ConvLSTM':
        model.add(Permute((2, 1, 3))) # for swap-dimension
        model.add(Reshape((-1,num_feat_map*dim)))
        model.add(LSTM(32, return_sequences=False, stateful=False))
        model.add(Dropout(0.5))
    if network_type == 'CNN':
        
        model.add(Flatten())
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.5))

        
def model_conv(model, num_feat_map):
    model.add(Conv2D(num_feat_map, kernel_size=(1, 5),
                 activation='relu',
                 input_shape=(dim, win_len, 1),
                 padding='same'))
    model.add(MaxPooling2D(pool_size=(1, 2)))
    model.add(Dropout(0.5))
    model.add(Conv2D(num_feat_map, kernel_size=(1, 5), activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(1, 2)))
    model.add(Dropout(0.5))
    
def model_LSTM(model):
    model.add(LSTM(num_hidden_lstm, 
               input_shape=(win_len,dim), 
               return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(num_hidden_lstm, return_sequences=False))
    model.add(Dropout(0.5))

def model_MLP(model, num_hidden_mlp):
    model.add(Dense(num_hidden_mlp, activation='relu', input_shape=(dim*win_len,)))
    model.add(Dropout(0.5))
    model.add(Dense(num_hidden_mlp, activation='relu'))
    model.add(Dropout(0.5))
    
def model_output(model):
    model.add(Dense(num_classes, activation='softmax'))
    

        



In [6]:
batch_size = 128
num_feat_map = 16
num_hidden_mlp = 128
num_hidden_lstm = 64

#network_type = 'CNN'
network_type = 'ConvLSTM'
#network_type = 'LSTM'
#network_type = 'MLP'
_, win_len, dim = X_train0.shape

X_train, X_valid = _data_reshaping(X_train0, X_valid0, network_type)


ConvLSTM


In [7]:
print(X_train.shape)

(9466, 52, 100, 1)


In [8]:

print('building the model ... ')
model = Sequential()


if network_type=='CNN' or network_type=='ConvLSTM':
    model_conv(model, num_feat_map)
    model_variant(model, num_feat_map, dim, network_type)
if network_type=='LSTM':
    model_LSTM(model)
if network_type=='MLP': 
    model_MLP(model, num_hidden_mlp) 
    
    
    
    
model_output(model)    
model.summary()




building the model ... 
ConvLSTM
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 52, 100, 16)       96        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 52, 50, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 52, 50, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 50, 16)        1296      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 52, 25, 16)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 52, 25, 16)        0         
_________________________________________________________________
permute_1 (Permute)          (None, 25, 52,

In [9]:
epochs = 3

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

H = model.fit(X_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            shuffle=True,
            validation_data=(X_valid, y_valid))


Train on 9466 samples, validate on 3478 samples
Epoch 1/3
9466/9466 [==============================] - 41s - loss: 1.3921 - acc: 0.5724 - val_loss: 0.8619 - val_acc: 0.7910
Epoch 2/3
9466/9466 [==============================] - 40s - loss: 0.6628 - acc: 0.8295 - val_loss: 0.5586 - val_acc: 0.8718
Epoch 3/3
9466/9466 [==============================] - 39s - loss: 0.4603 - acc: 0.8908 - val_loss: 0.4963 - val_acc: 0.8712


In [10]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

y_pred = np.argmax(model.predict(X_valid), axis=1)
y_true = np.argmax(y_valid, axis=1)
cf_matrix = confusion_matrix(y_true, y_pred)
print(cf_matrix)
class_wise_f1 = np.round(f1_score(y_true, y_pred, average=None)*100)*0.01
print('the mean-f1 score: {:.2f}'.format(np.mean(class_wise_f1)))


[[ 43   1   0   1   1   3   0   0   2   0   0   0]
 [  0 292   0   1   8   0   1   0   5   0   5   0]
 [  0   2 314   5   0   0   0   1   2   0   8   1]
 [  0   0  17 126   4   0   0   0  20   0  47  95]
 [  0   0   0   3 368   0   0   2   7   2   3   0]
 [  0   8   2   0   1 283   4  11   1   5   0   0]
 [  0   0   0   4   1   0 277  13   2   2   2   0]
 [  0   0   0   0   0   0   0 348   1   0   2   2]
 [  0   0   1   4   0   0   1   0 173   2   3   0]
 [  0   0   6   3   3   0   3   0  19 110  15   1]
 [  0   1   1   0   0   0   3   3  26   2 255  12]
 [  0   0   1   0   0   0   1   0   0   0  29 441]]
the mean-f1 score: 0.86


In [11]:
import time

start = time.time()
np.argmax(model.predict(X_valid), axis=1)
end = time.time()

elapsed = end - start
print('in millisecond')
print(1000*elapsed/len(X_valid))
#print(len(X_valid))

in millisecond
1.2644190017761616
